In [66]:
import src.login_config as config
import src.helpers as helpers
import src.data_scrape as ds

# username, password = config.main()

In [67]:
combined_df = helpers.read_csv_files('sold_listings')

In [70]:
combined_df['Listing link'][125]

'https://www.zoocasa.com/burnaby-bc-real-estate/eclipse-brentwood/1704-2381-beta-avenue'

In [69]:
browser = ds.selenium_start()
ds.first_time_login(browser)
lnks = ds.retrieve_sold_listing_description(browser, combined_df['Listing link'][0])

A


In [64]:
import importlib
importlib.reload(ds)

<module 'src.data_scrape' from 'C:\\Users\\Alan\\Projects\\Housing Project\\src\\data_scrape.py'>

In [33]:
# Key Facts:
    # Type: data-testid = TypeKeyFacts
    # Size: data-testid = SizeKeyFacts
    # Maintenance Fees (if applicable)
    # Approx Age: data-testid = Approx.AgeKeyFacts
    # MLS Number: data-testid = MLS®NumberKeyFacts
    # Levels: data-testid = LevelsKeyFacts
    # Garage: data-testid = GarageKeyFacts
    # Taxes: data-testid = TaxesKeyFacts
    # Average Price per sqft: data-test = Avg.PricePerSqftKeyFacts
    # Property Address: data-testid = keyFactsPropertyAddress
    # bedroom: data-testid = data-testid="listingBedIcon" .text
    # bathroom data-testid = listingBathIcon .text
# Sold Price: want listingStatus = sold
# List Price
# List Date
# End Date

# Rooms (if applicable)

# Description